In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Reference for EDA:
https://www.kaggle.com/ambrosm/tpsjan22-01-eda-which-makes-sense


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import random
import seaborn as sns
import re
import datetime
import math
from pathlib import Path
# from sklearn.model_selection import train_test_split

In [ ]:
train_data=pd.read_csv("../input/tabular-playground-series-jan-2022/train.csv")

In [ ]:
train_data.head(), train_data.tail()

In [ ]:
countries=set(train_data['country'])
countries

In [ ]:
products=set(train_data['product'])
products

In [ ]:
store_type=set(train_data['store'])
store_type

<h2> Gist of the the Data </h2>
<h4>
Row Ids <br>
Datetime Column indicating this is a Time Series Data <br>
Countries - Finland, Norway, Sweeden <br>
Store Types - Kaggle Mart and Kaggle Rama <br>
Products - Hats, Mugs, Stickers <br>
</h4>


<h2> EDA </h2>

In [ ]:
# Country -wise Sale 
train_data['country'].value_counts()
# Inference -> Same sales  in each of the Country

In [ ]:
# Product Wise Sale
train_data['product'].value_counts()
# Inference -> Same sales of each of the three Product

In [ ]:
# Store wise Sale
train_data['store'].value_counts()
# Inference -> Same sales at both the types of stores

In [ ]:
print('Train data dates ranging from:', train_data['date'].min(), '->', train_data['date'].max())

In [ ]:
train_data['date'] = pd.to_datetime(train_data['date'])

In [ ]:
train_data['Year'] = train_data.date.dt.year
train_data['Month'] = train_data.date.dt.month

Since this is a Time Series Forecasting Question the Test Dates would be dates in the 
future from 2019.

In [ ]:
# First let us Plot the Frequencies of Sales throughout the Time Frame
plt.figure(figsize=(9,9))
sns.lineplot(x = 'date', y = 'num_sold', data = train_data, label = 'daily')
# Inference -> December - January Festive season records the Highest amount of Sales
# April Easter Season gets second hgihest Spike in Sales

In [ ]:
plt.figure(figsize=(9,9))
sns.lineplot(x = 'date', y = 'num_sold', data = train_data,hue='store')
# Inference -> Kaggle RAMA Sales > Kaggle Mart Sales

In [ ]:
plt.figure(figsize=(9,9))
sns.lineplot(x = 'date', y = 'num_sold', data = train_data,hue='product')
# Inference ->  Sales of Hats > Sales of Mugs > Sales of Stickers

In [ ]:
plt.figure(figsize=(9,9))
sns.lineplot(x = 'date', y = 'num_sold', data = train_data,hue='country')
# Inference ->  Sales of Norway > Sales of Sweden > Sales of Finland

In [ ]:
plt.figure(figsize=(9,9))
sns.lineplot(x = 'Year', y = 'num_sold', data = train_data,hue='country')
# sns.lineplot(x = 'month', y = 'num_sold', data = train_data,hue='country')
# Inference ->  Sales of Norway > Sales of Sweden > Sales of Finland

In [ ]:
train_data.describe()
# Neglect it for row_id
# Daily Average for items Sold-> 388

In [ ]:
# Get a General trend of Increase in Sales, so the test data will definitely be an extrapolation
plt.figure(figsize=(9,9))
sns.lineplot(x = 'Year', y = 'num_sold', data = train_data,hue='store')

In [ ]:
#  Now we will Segregate the Data Country-wise and further do some Visualization before moving
# to Feature Engineering
train_data_Finland = train_data[train_data.country == "Finland"]
train_data_Norway = train_data[train_data.country == "Norway"]
train_data_Sweden = train_data[train_data.country == "Sweden"]

In [ ]:
# Now we will analyse the trends for each Country as Well
train_data_Finland['num_sold'].describe(),train_data_Norway['num_sold'].describe(),train_data_Sweden['num_sold'].describe()

In [ ]:
train_monthly_country = train_data.set_index('date').groupby([pd.Grouper(freq = 'M'), 'country'])[['num_sold']].mean()
plt.figure(figsize = (9, 9))
sns.lineplot(x = 'date', y = 'num_sold', hue = 'country', data = train_monthly_country)

In [ ]:
train_data.groupby(['country', 'store', 'product']).num_sold.agg(['max', 'mean'])

<h2> Data Cleaning and Feature Engineering

<h4>
So obviously there ia a Trend of Norway > Sweden > Finland <br>
So the Feature Engineering further will require a row of GDP Like how a lot of people have done <br>
So we will have to bring the GDP Dataset as an added feature for the Model <br>


In [ ]:
# Importing the Datasets for completeness
train=pd.read_csv('../input/tabular-playground-series-jan-2022/train.csv')
test=pd.read_csv('../input/tabular-playground-series-jan-2022/test.csv')
gdp=pd.read_csv('../input/gdp-20152019-finland-norway-and-sweden/GDP_data_2015_to_2019_Finland_Norway_Sweden.csv')
sample_sub=pd.read_csv('../input/tabular-playground-series-jan-2022/sample_submission.csv')

In [ ]:
train.head(),test.head(),gdp.head(),sample_sub.head()

In [ ]:
# Defining sMAPE
def SMAPE(y_true, y_pred):
    denominator = (y_true + np.abs(y_pred)) / 200.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return np.mean(diff)
def SMAPE2(y_true, y_pred):
    denominator = (y_true + np.abs(y_pred)) / 200.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return diff.squeeze()


In [ ]:
import holidays
import lightgbm
import pickle
from datetime import datetime
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GroupKFold
import dateutil.easter as easter

<h3> Reference for Feature Engineering and Hybrid Model </h3>
<h4> https://www.kaggle.com/teckmengwong/tps2201-hybrid-time-series


In [ ]:
# Feature engineering

from matplotlib.ticker import MaxNLocator, FormatStrFormatter, PercentFormatter
from scipy import stats
import ipywidgets as widgets
from learntools.time_series.style import *  # plot style settings
from learntools.time_series.utils import (create_multistep_example,
                                          load_multistep_data,
                                          make_lags,
                                          make_multistep_target,
                                          plot_multistep)

In [ ]:
# Importing Libs
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess


In [ ]:
# The Model that is trained is Hybrid Model GBM + XGboost + Linear
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import lightgbm as lgb

We have all the dependencies in place and Now we can start with Data Cleaning
and Feature Engineering

In [ ]:
# Handling the Missing Values
def handle_missing_values(df):
    # Any numerical data must be replaced by 0 
    for data in df.select_dtypes("number"):
        df[data] = df[data].fillna(0)
    # Categorical Data must be replaced by None
    for data in df.select_dtypes("category"):
        df[data] = df[data].fillna("None")
    return df


In [ ]:
def fourier_features(index, freq, order):
    time = np.arange(len(index), dtype=np.float32)
    k = 2 * np.pi * (1 / freq) * time
    features = {}
    for i in range(1, order + 1):
        features.update({
            f"sin_{freq}_{i}": np.sin(i * k),
            f"cos_{freq}_{i}": np.cos(i * k),
        })
    return pd.DataFrame(features, index=index)

In [ ]:
# time series data common new feature  
DATE = "date"
YEAR = "year"
MONTH = "month"
WEEK = "week"
DAY = "day"
DAYOFYEAR = "dayofyear"
DAYOFMONTH = "dayofMonth"
DAYOFWEEK = "dayofweek"
WEEKDAY = "weekday"

Feature Engineering Ref: https://www.kaggle.com/teckmengwong/tps2201-hybrid-time-series#Fine-tuning

In [ ]:
def get_basic_ts_features(df):
    
    gdp.set_index('year', inplace=True)
    
    def get_gdp(row):
        country = 'GDP_' + row.country
        return gdp_df.loc[row.date.year, country]
    
    df['gdp'] = np.log(df.apply(get_gdp, axis=1))
    
    for country in ['Finland', 'Norway']:
        df[country] = df.country == country
    for store in ['KaggleMart']:
        df[store] = df['store'] == store
    for product in ['Kaggle Mug', 'Kaggle Sticker']:
        df[product] = df['product'] == product
    

    df[WEEKDAY] = df[DATE].dt.weekday == 4
    df[WEEKDAY] = df[DATE].dt.weekday >= 5
    
    # 21 days cyclic for lunar
    dayofyear = df.date.dt.dayofyear
    for k in range(1, 22, 1):
        df[f'sin{k}'] = np.sin(dayofyear / 365 * 2 * math.pi * k)
        df[f'cos{k}'] = np.cos(dayofyear / 365 * 2 * math.pi * k)
        df[f'Finland_sin{k}'] = df[f'sin{k}'] * df['Finland']
        df[f'Finland_cos{k}'] = df[f'cos{k}'] * df['Finland']
        df[f'Norway_sin{k}'] = df[f'sin{k}'] * df['Norway']
        df[f'Norway_cos{k}'] = df[f'cos{k}'] * df['Norway']
        df[f'store_sin{k}'] = df[f'sin{k}'] * df['KaggleMart']
        df[f'store_cos{k}'] = df[f'cos{k}'] * df['KaggleMart']
        df[f'mug_sin{k}'] = df[f'sin{k}'] * df['Kaggle Mug']
        df[f'mug_cos{k}'] = df[f'cos{k}'] * df['Kaggle Mug']
        df[f'sticker_sin{k}'] = df[f'sin{k}'] * df['Kaggle Sticker']
        df[f'sticker_cos{k}'] = df[f'cos{k}'] * df['Kaggle Sticker']
    
    # End of year - Christmas Season
    for d in range(24, 32):
        df[f"dec{d}"] = (df.date.dt.month == 12) & (df.date.dt.day == d)
    # Start of the new Year
    for d in range(1, 13):
        df[f"jan{d}"] = (df.date.dt.month == 1) & (df.date.dt.day == d)
    # May
    for d in list(range(1, 10)) + list(range(17, 25)):
        df[f"may{d}"] = (df.date.dt.month == 5) & (df.date.dt.day == d)
    # June
    for d in list(range(6, 14)):
        df[f"june{d}"] = (df.date.dt.month == 6) & (df.date.dt.day == d)
    
    # Last Wednesday of June
    wed_june_date = df.date.dt.year.map({2015: pd.Timestamp(('2015-06-24')),
                                         2016: pd.Timestamp(('2016-06-29')),
                                         2017: pd.Timestamp(('2017-06-28')),
                                         2018: pd.Timestamp(('2018-06-27')),
                                         2019: pd.Timestamp(('2019-06-26'))})
    for d in list(range(-5, 6)):
        df[f"wed_june{d}"] = (df.date - wed_june_date == np.timedelta64(d, "D"))
        
    # First Sunday of November
    sun_nov_date = df.date.dt.year.map({2015: pd.Timestamp(('2015-11-1')),
                                         2016: pd.Timestamp(('2016-11-6')),
                                         2017: pd.Timestamp(('2017-11-5')),
                                         2018: pd.Timestamp(('2018-11-4')),
                                         2019: pd.Timestamp(('2019-11-3'))})
    for d in list(range(0, 10)):
        df[f"sun_nov{d}"] = (df.date - sun_nov_date == np.timedelta64(d, "D"))
        
    # Easter
    easter_date = df.date.apply(lambda date: pd.Timestamp(easter.easter(date.year)))
    for d in list(range(0, 9)) + list(range(50, 60)) + list(range(40, 46)):
        df[f"easter{d}"] = (df.date - easter_date == np.timedelta64(d, "D"))
                                      
#     df.drop(columns=[DATE], inplace = True)
    
    return df

In [ ]:
target_column = train.columns.difference(
    test.columns)[0]  


In [ ]:
train[DATE] = pd.to_datetime(train[DATE])
test[DATE] = pd.to_datetime(test[DATE])

In [ ]:
# Feature Engineering
train = get_basic_ts_features(train)
test = get_basic_ts_features(test)

In [ ]:
train.head(),test.head()

In [ ]:
def encoding_func(data):
    output = data.copy()
     
    for col in data.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        
#     for colname,col in output.iteritems():
#                 output[colname] = LabelEncoder().fit_transform(col)
    return output

In [ ]:
train_1=encoding_func(train)
train_1

In [ ]:
test_1=encoding_func(test)

In [ ]:
X = train_1.set_index([DATE]).sort_index()
X_test = test.set_index([DATE]).sort_index()

In [ ]:
train_data_final = train.set_index(['date', 'country', 'store', 'product']).sort_index()

In [ ]:
#  Last Cell
# sub['num_sold'] = np.round(sub['num_sold']).astype(int)
# import pandas as pd
# sub = pd.read_csv('../input/tps-jan-2022-automated-ensembling/submission.csv')
# sub.num_sold *= 0.99
# sub.to_csv('submission.csv', index=False)
# sub